#### importer les libs

In [206]:
import pandas as pd
import sklearn as sk
from sklearn import model_selection
from sklearn import linear_model

#### déclarer les classes

In [207]:
class DataHandler:
    def __init__(self):
        print("\nDataHandler: __init__")
        self.price = None
        self.listing = None
        self.merge = None
        ;

    def get_data(self):
        print("\nDataHandler: get_data")
        self.price = pd.read_csv("/Users/thomas/Downloads/price_availability.csv", sep=";")
        self.listing = pd.read_csv("/Users/thomas/Downloads/listings_final.csv", sep=";")

    def group_data(self):
        print("\nDataHandler: group_data")
        self.listing.drop("Unnamed: 0", axis="columns", inplace=True)
        mean = self.price.groupby("listing_id")['local_price'].mean()
        self.merge = self.listing.merge(mean, on="listing_id")

    def get_process_data(self):
        print("\nDataHandler: get_process_data")
        self.get_data()
        self.group_data()
        return self.merge
        ;

class FeatureRecipe:
    """
    Feature processing class
    """
    def __init__(self, data: pd.DataFrame):
        print("\nFeatureRecipe: __init__")
        self.data = data
        self.continuous = []
        self.categorical = []
        self.discrete = []
        self.datetime = []

    def separate_variable_types(self) -> None:
        print("\nFeatureRecipe: separate_variable_types")
        """ TODO : Diviser les types de variables dans un tableau """
        for lbl in self.data.columns:
            c = self.data[lbl]
            if c.dtype == int:
                self.discrete.append(lbl)
            elif c.dtype == float:
                self.continuous.append(lbl)
        pass

    def drop_uselessf(self):
        print("\nFeatureRecipe: drop_uselessf")
        """ TODO : Supprimer les colonnes inutiles du dataset """
        print("nb columns before: {}".format(self.data.columns.size))
        drop = ["name", "neighborhood", "latitude", "longitude", "city"]
        self.data.drop(axis="columns", labels=drop, inplace=True)
        print("nb columns after: {}, dropped: {}".format(self.data.columns.size, drop))
        pass

    def deal_duplicate(self):
        print("\nFeatureRecipe: deal_duplicate")
        """ TODO : Supprimer les colonnes dupliquées du dataset """
        print("nb columns before: {}".format(self.data.columns.size))
        drop = []
        i = 1
        for lbl in self.data.columns:
            c = self.data[lbl]
            for lbl2 in self.data.columns[i:]:
                c2 = self.data[lbl2]
                if c.equals(c2):
                    drop.append(lbl2)
                    break
            i+=1

        self.data.drop(columns=drop, inplace=True)
        print("nb columns after: {}, dropped = {}".format(self.data.columns.size, drop))
        pass

    """ Supprimer un pourcentage de NA du dataset """
    def drop_nanp(self, threshold: float):
        print("\nFeatureRecipe: drop_nanp", threshold)
        print("nb columns before: {}".format(self.data.columns.size))
        drop = []
        for lbl in self.data.columns:
            c = self.data[lbl]
            if c.isna().sum() / self.data.shape[0] >= threshold:
                drop.append(lbl)
        self.data.drop(columns=drop, inplace=True)
        print("nb columns after: {}, dropped: {}".format(self.data.columns.size, drop))
        pass

    def deal_dtime(self):
        print("\nFeatureRecipe: deal_dtime")
        """ TODO : Traiter les DateTime """
        pass

    def prepare_data(self, threshold: float):
        print("\nFeatureRecipe: prepare_data")
        self.drop_uselessf()
        self.deal_duplicate()
        self.drop_nanp(threshold)
        self.deal_dtime()
        self.separate_variable_types()

#### commencer le processing

In [208]:
d = DataHandler()
listing_price_mean = d.get_process_data()
%time


DataHandler: __init__

DataHandler: get_process_data

DataHandler: get_data

DataHandler: group_data
CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.81 µs


In [209]:
fr = FeatureRecipe(listing_price_mean)
fr.prepare_data(0.30)
%time


FeatureRecipe: __init__

FeatureRecipe: prepare_data

FeatureRecipe: drop_uselessf
nb columns before: 19
nb columns after: 14, dropped: ['name', 'neighborhood', 'latitude', 'longitude', 'city']

FeatureRecipe: deal_duplicate
nb columns before: 14
nb columns after: 13, dropped = ['is_business_travel_ready']

FeatureRecipe: drop_nanp 0.3
nb columns before: 13
nb columns after: 13, dropped: []

FeatureRecipe: deal_dtime

FeatureRecipe: separate_variable_types
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs
